In [2]:
import pickle
import pandas as pd
import os
import numpy as np
import torch

In [3]:
result_folder = 'results'
os.makedirs(result_folder, exist_ok=True)

for i in range(1, 16):
    subfolder_name = f'vp{i}'
    os.makedirs(os.path.join(result_folder, subfolder_name), exist_ok=True)

In [8]:
result_folder = 'csv'
os.makedirs(result_folder, exist_ok=True)

In [4]:
annotations_converter = pickle.load(open("/content/annotation_converter.pkl", "rb"))
annotations_csv_path = "/content/midlevel.chunks_90.csv"
df_annotations = pd.read_csv(annotations_csv_path, delimiter=',')

In [5]:
activity_index_map = {activity: index for index, activity in enumerate(annotations_converter)}
df_annotations['activity_index'] = df_annotations['activity'].map(activity_index_map)

In [9]:
for root, _, files in os.walk("/content/results"):
    for file in files:
        folder_name = os.path.basename(root)
        file_path = f"/content/results/{folder_name}/{file}"
        tensors = torch.load(file_path)
        filtered_annotations = df_annotations[
            df_annotations["file_id"]
            == f"{folder_name}/{os.path.splitext(file)[0]}"
        ]
        flattened_data = []
        max_frame = filtered_annotations["frame_end"].max()
        tensors_len = len(tensors)
        for _, row in filtered_annotations.iterrows():
            if pd.notna(row["activity_index"]):
                for frame in range(row["frame_start"], row["frame_end"] + 1):
                    if frame < tensors_len:
                        flattened_data.append(
                            {
                                "frame": frame,
                                "activity": row["activity"],
                                "activity_index": int(row["activity_index"]),
                                "predicted": annotations_converter[int(np.argmax(tensors[frame]))],
                                "predicted_index": np.argmax(tensors[frame]),
                            }
                        )
        flattened_df = pd.DataFrame(flattened_data)
        flattened_df.to_csv(f'/content/csv/{folder_name}-{os.path.splitext(file)[0]}.csv', index=False)
        matches = flattened_df['activity_index'] == flattened_df['predicted_index']
        accuracy = matches.sum() / len(flattened_df) * 100
        print(f"Accuracy for '{folder_name}/{os.path.splitext(file)[0]}' is {accuracy:.2f}%")

<ipython-input-9-96bb50ac57a6>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensors = torch.load(file_path)


Accuracy for 'vp2/run1_2018-05-03-14-08-31.kinect_color' is 89.39%
Accuracy for 'vp2/run2_2018-05-24-17-22-26.kinect_color' is 88.12%
